# Import master data/conditions files,

In [1]:
#Import the necessary libraries
import pandas as pd
import numpy as np
import random

In [2]:
#Install Faker to enable generation of random customer names
!pip install faker

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 41.3 MB/s eta 0:00:00


In [3]:
from faker import Faker
faker = Faker()
Faker.seed(55)

In [4]:
# Upload the MasterData and Conditions file and set the path to the master data file and conditions file
master_data_path = "MasterData.xlsx"
conditions_data_path = "Conditions.xlsx"

# Load all sheets in both the files
master_data = pd.read_excel(master_data_path, sheet_name=None)
conditions_data = pd.read_excel(conditions_data_path, sheet_name=None)

# Print the sheet names
print("Master Data Sheets:", list(master_data.keys()))
print("Conditions Data Sheets:", list(conditions_data.keys()))

Master Data Sheets: ['Customer Information', 'Account Information', 'Transaction data', 'Business', 'Geography', 'Customer Type', 'Product', 'Channels', 'TranType', 'Currency', 'watch list data']
Conditions Data Sheets: ['Prod-TranType Mapping', 'Channe-TranType Mapping']


In [5]:
#Extract each masterdata excel sheet into a separate dataframe
business_master = master_data["Business"]
currency_master = master_data["Currency"]
geo_master = master_data["Geography"]
ctype_master = master_data["Customer Type"]
prod_master = master_data["Product"]
chann_master = master_data["Channels"]
tran_master = master_data["TranType"]

In [ ]:
#Check the business_master dataframe
business_master

,BusinessCode,BusinessDesc,RiskRating
0,1001,Food,LR
1,1002,Textiles,LR
2,1003,Machinary and Equipment,LR
3,1004,Chemicals,LR
4,1005,Electronics,LR
5,1006,Metal & Metal Products,LR
6,1007,Retail,LR
7,1008,Information Tech,LR
8,1009,Construction,LR
9,1010,Transport,MR


In [ ]:
#Check the geo_master dataframe
geo_master

,GeoCode,GeoDesc,RiskRating
0,CAN,Canada,LR
1,AUT,Austria,LR
2,COL,Colambia,LR
3,BRA,Brazil,LR
4,NPL,Nepal,HR
5,IND,India,LR
6,USD,United States of America,LR
7,ARE,UAE,LR
8,AUS,Australia,LR
9,HKG,Hongkong,LR


# Generate customer data

In [6]:
#Define a function to ensure the synthetic customer generation has a realistic mix of High Risk, Med Risk and Low risk master data (Business Type, Geography, Customer Type)
#Set thresholds for the same High Risk-0.3%, Med Risk-0.5% and Low risk-99.7% for Business type for example
def sample_with_risk_distribution(master_df, risk_col, code_col, target_dist, total_needed):
    samples = []

    for risk_level, fraction in target_dist.items():
        subset = master_df[master_df[risk_col] == risk_level]
        n_samples = int(total_needed * fraction)

        if len(subset) == 0:
            continue  # Skip if no rows with this risk rating

        sampled = subset.sample(n=n_samples, replace=(n_samples > len(subset)))
        samples.append(sampled[[code_col]])

    return pd.concat(samples, ignore_index=True)

# Desired distributions
business_risk_dist = {"HR": 0.003, "MR": 0.005, "LR": 0.992}
geo_risk_dist = {"HR": 0.005, "MR": 0.005, "LR": 0.99}
cust_type_risk_dist = {"HR": 0.003, "MR": 0.01, "LR": 0.992}

num_customers = 500

# Sample for business
business_sample = sample_with_risk_distribution(
    business_master, "RiskRating", "BusinessCode", business_risk_dist, num_customers
)

geo_sample = sample_with_risk_distribution(
    geo_master, "RiskRating", "GeoCode", geo_risk_dist, num_customers
)

cust_type_sample = sample_with_risk_distribution(
    ctype_master, "RiskRating", "CustomerTypeCode", cust_type_risk_dist, num_customers)


In [7]:
# Generate 500 customers
# Add a meaningful suffix with the names of the customers based on customer type
start_id = 2001
customers = []
for i in range(500):
    cust_type = cust_type_sample.iloc[i % len(cust_type_sample)]["CustomerTypeCode"]
    name = faker.name()
    suffix_map = {
    "PUBL": " Pub Ltd Company",
    "PVTL": " Pvt Ltd Company",
    "GOVT": " Govt Co",
    "SPRF": " Sole Prop firm",
    "PART": " and Partners",
    "LLPF": " Limited LP",
    "TRST": " Trust Co",
    "CLSO": " Club",
    "NBFC": " Non-Banking Fin Co",
    "BFIS": " Banking Ltd",
    "SGOV": " State Gov Co",
    "ASSO": " & Association",
    "MBNK": " Banking Ltd",
    "NGOS": " Non-Govt Org",
    "FCOM": " Foreign Co"
     }
    suffix = suffix_map.get(cust_type, "")
    name += suffix
    cust = {
        "CustomerID": f"CUST{start_id+i:04}",
        "Name": name,
        "PEP_Flag": 'Y' if np.random.rand() < 0.02 else 'N',
        "Business": business_sample.iloc[i % len(business_sample)]["BusinessCode"],
        "Geography": geo_sample.iloc[i % len(geo_sample)]["GeoCode"],
        "CustomerType": cust_type
    }
    customers.append(cust)


df_customers = pd.DataFrame(customers)

In [8]:
#Check the structure of the customers dataframe
df_customers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   CustomerID    500 non-null    object
 1   Name          500 non-null    object
 2   PEP_Flag      500 non-null    object
 3   Business      500 non-null    int64 
 4   Geography     500 non-null    object
 5   CustomerType  500 non-null    object
dtypes: int64(1), object(5)
memory usage: 23.6+ KB


In [9]:
#Check the sample data in the customers dataframe
df_customers.head()

,CustomerID,Name,PEP_Flag,Business,Geography,CustomerType
0,CUST2001,Trevor Hernandez Club,N,1014,NPL,CLSO
1,CUST2002,Sarah Burgess Sole Prop firm,N,1012,NPL,SPRF
2,CUST2003,Guy Bowen Sole Prop firm,N,1023,KEN,SPRF
3,CUST2004,Mary Fernandez Sole Prop firm,N,1001,ZAF,SPRF
4,CUST2005,Jared Ball Sole Prop firm,N,1002,BHR,SPRF


In [ ]:
#Download the synthetic customers dataframe to a csv file
df_customers.to_csv("customers.csv", index=False)

# Generate account Data

In [10]:
#Define a function to ensure the synthetic accounts data generation has a realistic mix of High Risk, Med Risk and Low risk master data (Product Type and Channel)
#Set thresholds for the same High Risk-0.5%, Med Risk-1% and Low risk-98.5% for Product for example
product_risk_dist = {"HR": 0.005, "MR": 0.01, "LR": 0.985}
currency_risk_dist = {"HR": 0.005, "MR": 0.01, "LR": 0.985}
num_accounts = 1100

# Sample products and currencies as per risk split
product_sample = sample_with_risk_distribution(
prod_master, "RiskRating", "ProductCode", product_risk_dist, num_accounts
)

currency_sample = sample_with_risk_distribution(
    currency_master, "RiskRating", "CurrencyCode", currency_risk_dist, num_accounts
)

In [11]:
#Define thresholds for synthetic generation based on currency, for example 70% of the accounts to be in USD currency, to reflect a realistic composition of accounts
num_accounts = 1100
num_customers = len(df_customers)
assert num_accounts >= num_customers, "Number of accounts must be >= number of customers"

# Phase 1: One account per customer
accounts = []
start_id = 2001
currency_list = currency_master["CurrencyCode"].tolist()

explicit_weights = {
    "USD": 0.7,
    "EUR": 0.1,
    "GBP": 0.05,
    "CAD": 0.03,
    "INR": 0.02,
    "JPY": 0.02,
    "AUD": 0.02,
}

def choose_currency_weighted(currency_list, weights_dict):
    weights = [weights_dict.get(curr, 0.06 / (len(currency_list) - len(weights_dict)))
               if curr not in weights_dict else weights_dict[curr]
               for curr in currency_list]
    return random.choices(currency_list, weights=weights, k=1)[0]

# Sample products and currencies upfront
product_sample = sample_with_risk_distribution(
    prod_master, "RiskRating", "ProductCode", product_risk_dist, num_accounts
)
currency_sample = sample_with_risk_distribution(
    currency_master, "RiskRating", "CurrencyCode", currency_risk_dist, num_accounts
)

for i, cust_id in enumerate(df_customers["CustomerID"]):
    acc = {
        "AccountNumber": f"ACC{start_id+i:05}",
        "CustomerID": cust_id,
        "ProductCode": product_sample.iloc[i % len(product_sample)]["ProductCode"],
        "CurrencyCode": choose_currency_weighted(currency_list, explicit_weights),
        "Balance": round(np.random.uniform(1000, 1000000), 2)
    }
    accounts.append(acc)

# Phase 2: Remaining random accounts
remaining_accounts = num_accounts - num_customers
for i in range(remaining_accounts):
    acc = {
        "AccountNumber": f"ACC{num_customers + start_id+i + 1:05}",
        "CustomerID": np.random.choice(df_customers["CustomerID"]),
        "ProductCode": product_sample.iloc[(num_customers + i) % len(product_sample)]["ProductCode"],
        "CurrencyCode": choose_currency_weighted(currency_list, explicit_weights),
        "Balance": round(np.random.uniform(1000, 1000000), 2)
    }
    accounts.append(acc)

df_accounts = pd.DataFrame(accounts)

In [12]:
#Check the structure of the accounts dataframe
df_accounts.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1100 entries, 0 to 1099
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   AccountNumber  1100 non-null   object 
 1   CustomerID     1100 non-null   object 
 2   ProductCode    1100 non-null   object 
 3   CurrencyCode   1100 non-null   object 
 4   Balance        1100 non-null   float64
dtypes: float64(1), object(4)
memory usage: 43.1+ KB


In [13]:
#Check the sample data in the accounts dataframe
df_accounts.head()

,AccountNumber,CustomerID,ProductCode,CurrencyCode,Balance
0,ACC02001,CUST2001,BUSL,USD,468616.64
1,ACC02002,CUST2002,BUSL,GBP,21364.21
2,ACC02003,CUST2003,BUSL,AUD,233323.92
3,ACC02004,CUST2004,BUSL,SGD,985716.43
4,ACC02005,CUST2005,BUSL,USD,881674.27


In [ ]:
#Download the synthetic accounts dataframe to a csv file
df_accounts.to_csv("accounts.csv", index=False)

# Generate transactions data

In [14]:
#Define a function to set thresolds based on channel, currency, geography and transaction type
def weighted_sample(df, rating_col, code_col, total=10000, weights={"HR":0.005, "MR":0.005, "LR":0.99}):
    samples = []
    for risk, pct in weights.items():
        subset = df[df[rating_col] == risk]
        if not subset.empty:
            count = int(total * pct)
            samples.extend(subset.sample(n=count, replace=True)[code_col].tolist())
    return samples

tran_types = weighted_sample(tran_master, "RiskRating", "TranTypeCode")
channels = weighted_sample(chann_master, "RiskRating", "ChannelCode", total=len(tran_types))
currencies = weighted_sample(currency_master, "RiskRating", "CurrencyCode", total=len(tran_types))
geos = weighted_sample(geo_master, "RiskRating", "GeoCode", total=len(tran_types))

In [15]:
#Import the conditions of allowed combinations (Product-Transaction type and Product-Channel) in transactions
prod_tran_map = conditions_data["Prod-TranType Mapping"]
channel_tran_map = conditions_data["Channe-TranType Mapping"]

In [ ]:
#Check the allowed combination for Channel-TranType mapping in the dataframe
channel_tran_map

,ChannelCode,Channel Name,TranTypeCode
0,BRAN,Branch,"CDEP, CWIT,FTDO,FTIN, LDIST"
1,CATM,ATM,"CDEP, CWIT,FEEC"
2,CPOS,POS,"MPAY, FEEC,LREP"
3,IBAN,Internet Banking,"FTDO, FTIN, CCPY,MPAY, LREP, BPAY, FEEC"
4,MBAN,Mobile Banking,"FTDO, FTIN, CCPY,MPAY, LREP, BPAY, FEEC"
5,CUPI,UPI,"CDEP, CWIT, LREP"
6,CHEQ,Cheque / DD,"FTDO,CCPY, LDIST,LREP,BPAY,FEEC"
7,CARD,Cards,"FTDO, FTIN, CCPY,MPAY, FEEC"
8,WALL,Wallets,"FTDO, FTIN, CCPY,MPAY,LDIST"
9,CUST,Customer Terminal,"FTDO, FTIN, CCPY,MPAY,LREP"


In [ ]:
#Check the allowed combination for Product-TranType mapping in the dataframe
prod_tran_map

,ProductCode,Prod Name,TranTypeCode
0,FIXD,Fixed Deposit,"CDEP, CWIT, FTDO, BPAY, FEEC"
1,BUSL,Business Loans,"CDEP, CWIT, FTDO, FTIN,MPAY,CCPY, LDIST,LREP,B..."
2,SAVS,Savings,"CDEP, CWIT, CCPAY, FTDO, FTIN"
3,CHEK,Checking,"CDEP, CWIT, CCPAY, FTDO, FTIN, BPAY"
4,OVDF,Overdraft,"CDEP, CWIT, FTDO, FTIN,MPAY,CCPY"
5,AUTL,Auto Loan,"FTDO, CDEP, FEEC"
6,HOUL,Housing Loan,"FTDO, CDEP, FEEC"
7,BILLS,Bills Discounting,"CDEP, CWIT, CCPAY, FTDO, FTIN, BPAY"
8,GLON,Gold Loans,"CDEP, CWIT, FTDO, FTIN,MPAY,CCPY, LDIST,LREP,B..."
9,ELON,Education Loans,"CDEP, CWIT, FTDO, FTIN,MPAY,CCPY"


In [16]:
#Seperate the allowed transaction types (which were mapped with a ',')
normalized_prod_tran_map = (
    prod_tran_map.assign(TranTypeCode=prod_tran_map["TranTypeCode"].str.split(","))
                 .explode("TranTypeCode")
)
normalized_prod_tran_map["TranTypeCode"] = normalized_prod_tran_map["TranTypeCode"].str.strip()

normalized_chan_tran_map = (
    channel_tran_map.assign(TranTypeCode=channel_tran_map["TranTypeCode"].str.split(","))
                 .explode("TranTypeCode")
)
normalized_chan_tran_map["TranTypeCode"] = normalized_chan_tran_map["TranTypeCode"].str.strip()

In [17]:
#Define the exchange rates to be used to convert the transaction amount to 'amount in local currency(LCY)'

exchange_rates = {
    "USD": 1.0,
    "EUR": 1.1,
    "GBP": 1.25,
    "SGD": 0.78,
    "INR": 0.012,
    "VEF": 0.08,
    "CAD": 0.73,
    "JPY": 0.0068,
    "ZAR": 0.056,
    "CNY": 0.14,
    "DKK": 0.15,
    "EGP": 0.021,
    "AUD": 0.65
}

In [18]:
#Generate 11000 synthetic transactions with (a) thresolds on the currency of the transaction (b)thresolds on the amount of the transactions
from collections import defaultdict
num_transactions=11000
start_id = 20000
transactions = []
valid_combinations = []  # (TranType, Channel, Product)
for _, row in normalized_prod_tran_map.iterrows():
    tran_types = [t.strip() for t in row["TranTypeCode"].split(",")]
    product = row["ProductCode"]
    for tran in tran_types:
        allowed_channels = normalized_chan_tran_map[normalized_chan_tran_map["TranTypeCode"] == tran]["ChannelCode"].tolist()
        for channel in allowed_channels:
            valid_combinations.append((tran, channel, product))

currency_list = currency_master["CurrencyCode"].tolist()
explicit_weights = {
    "USD": 0.7,
    "EUR": 0.1,
    "GBP": 0.05,
    "CAD": 0.03,
    "INR": 0.02,
    "JPY": 0.02,
    "AUD": 0.02
}

def choose_currency_weighted(currency_list, explicit_weights, other_weight=0.06):
    all_weights = {}
    others = [c for c in currency_list if c not in explicit_weights]

    # Distribute 'others' weight equally
    if others:
        per_other_weight = other_weight / len(others)
        for c in others:
            all_weights[c] = per_other_weight

    # Add explicit weights
    all_weights.update(explicit_weights)

    # Normalize and sample
    currencies = list(all_weights.keys())
    weights = list(all_weights.values())
    return random.choices(currencies, weights=weights, k=1)[0]

amount_slabs = [
    (50, 1000),
    (1001, 5000),
    (5001, 20000),
    (20001, 100000)
]
slab_weights = [0.4, 0.3, 0.2, 0.1]  # Should sum to 1.0

def generate_transaction_amount(slabs, weights):
    selected_slab = random.choices(slabs, weights=weights, k=1)[0]
    return round(random.uniform(selected_slab[0], selected_slab[1]), 2)

# Now generate transactions using these valid combinations

for i in range(num_transactions):
    tran_type, channel, product = random.choice(valid_combinations)
    #amount = round(np.random.uniform(50, 100000), 2)
    amount = generate_transaction_amount(amount_slabs, slab_weights)
    #currency = random.choice(currency_master["CurrencyCode"].tolist())
    currency = choose_currency_weighted(currency_list, explicit_weights)
    exchange_rate = exchange_rates.get(currency, 1.0)  # default to 1.0 if not found
    tlcy_amount = round(amount * exchange_rate, 2)

    start_date = pd.to_datetime("2025-08-03")
    end_date = pd.to_datetime("2025-08-16")
    date_range = pd.date_range(start=start_date, end=end_date).to_list()

    # Choose initiating and counterparty customers
    initiating = df_customers.sample(1).iloc[0]
    counterparty = df_customers.sample(1).iloc[0]
    while counterparty["CustomerID"] == initiating["CustomerID"]:
        counterparty = df_customers.sample(1).iloc[0]

    initiating = df_customers.sample(1).iloc[0]
    # get associated account
    initiating_accounts = df_accounts[df_accounts["CustomerID"] == initiating["CustomerID"]]
    initiating_account_id = initiating_accounts.sample(1).iloc[0]["AccountNumber"] if not initiating_accounts.empty else "Unknown"

    # Set geography rules for FTDO/FTIN
    if tran_type == "FTDO":
        counter_geo = initiating["Geography"]
    elif tran_type == "FTIN":
        counter_geo = random.choice([g for g in geo_master["GeoCode"] if g != initiating["Geography"]])
    else:
        counter_geo = counterparty["Geography"]

    transaction = {
        "TransactionDate": random.choice(date_range).strftime("%Y-%m-%d"),
        #"TransactionID": f"T{i+1:05d}",
        "TransactionID": f"T{start_id+i:05d}",
        "InitiatingCustomer": initiating["CustomerID"],
        "AccountNumber": initiating_account_id,
        "Counterparty": counterparty["CustomerID"],
        "TranType": tran_type,
        "Channel": channel,
        "Currency": currency,
        "Credit/Debit": random.choice(["Cr", "Dr"]),
        "Amount": amount,
        "Amount in LCY":tlcy_amount,
        "Product": product,
        "InitiatingGeo": initiating["Geography"],
        "CounterpartyGeo": counter_geo
        #"Timestamp": faker.date_time_between(start_date='-30d', end_date='now')
    }
    transactions.append(transaction)


In [19]:
#Save the transactions file in a csv format
transaction_df = pd.DataFrame(transactions)
#transaction_df.to_csv("transactions.csv", index=False)
print("✅ Synthetic transaction dataset created with", len(transaction_df), "rows.")

✅ Synthetic transaction dataset created with 11000 rows.


# Generate watchlist data

In [20]:
# Generate Watchlist by randomly marking 10 customers as watchlisted
watchlist_customers = df_customers.sample(10)
df_watchlist_cust = watchlist_customers[["Name"]].copy()
df_watchlist_cust["WatchReason"] = "FATF"
#df_watchlist_cust.to_csv("watchlist_cust.csv", index=False)

# Enrich transaction data with other relevant columns

In [21]:
accounts = df_accounts
customers = df_customers
transactions = transaction_df
watchlist = df_watchlist_cust

In [22]:
#Add other relevant columns to the base transaction file, by merging with the relevant master data file
#Other relevant fields-customer related: BusinessCode, GeoCode,Customer Type (along with the Risk levels for these fields), PEP Flag and Name of both Initiating customer and Counterparty
#Other relevant fields-transaction related: Channel, Product, Transaction Type (along with the Risk levels for these fields)
transactions_merge = transactions.merge(
    customers.set_index("CustomerID")[["Business","CustomerType","PEP_Flag","Name"]],
    left_on="InitiatingCustomer",
    right_index=True,
    how="left"
).rename(columns={"Business": "BusinessCode_IC",
                  "CustomerType": "CustTypeCode_IC",
                  "Name":"ICName",
                  "PEP_Flag":"PEP_Flag_IC"})

transactions_merge = transactions_merge.merge(
    customers.set_index("CustomerID")[["Business","CustomerType","PEP_Flag","Name"]],
    left_on="Counterparty",
    right_index=True,
    how="left"
).rename(columns={"Business": "BusinessCode_CP",
                  "CustomerType": "CustTypeCode_CP",
                  "Name":"CounterpartyName",
                  "PEP_Flag":"PEP_Flag_CP"})

transactions_merge = transactions_merge.merge(accounts.set_index("AccountNumber")[["ProductCode"]],
                                        on="AccountNumber",
                                        how='left')

transactions_merge = transactions_merge.merge(
    business_master.set_index("BusinessCode")["RiskRating"],
    left_on="BusinessCode_IC",
    right_index=True,
    how="left"
).rename(columns={"RiskRating": "IC_BusinessRisk"})

transactions_merge = transactions_merge.merge(
    business_master.set_index("BusinessCode")["RiskRating"],
    left_on="BusinessCode_CP",
    right_index=True,
    how="left"
).rename(columns={"RiskRating": "CP_BusinessRisk"})


transactions_merge = transactions_merge.merge(
    chann_master.set_index("ChannelCode")["RiskRating"],
    right_index=True,
    left_on ="Channel",
    how="left"
).rename(columns={"RiskRating": "ChannelRisk"})

transactions_merge = transactions_merge.merge(
    geo_master.set_index("GeoCode")["RiskRating"],
    right_index=True,
    left_on ="InitiatingGeo",
    how="left"
).rename(columns={"RiskRating": "IC_GeoRisk"})

transactions_merge = transactions_merge.merge(
    geo_master.set_index("GeoCode")["RiskRating"],
    right_index=True,
    left_on ="CounterpartyGeo",
    how="left"
).rename(columns={"RiskRating": "CP_GeoRisk"})

transactions_merge = transactions_merge.merge(
    ctype_master.set_index("CustomerTypeCode")["RiskRating"],
    right_index=True,
    left_on ="CustTypeCode_IC",
    how="left"
).rename(columns={"RiskRating": "IC_CustTypeRisk"})

transactions_merge = transactions_merge.merge(
    ctype_master.set_index("CustomerTypeCode")["RiskRating"],
    right_index=True,
    left_on ="CustTypeCode_CP",
    how="left"
).rename(columns={"RiskRating": "CP_CustTypeRisk"})


transactions_merge = transactions_merge.merge(
    prod_master.set_index("ProductCode")["RiskRating"],
    right_index=True,
    left_on ="Product",
    how="left"
).rename(columns={"RiskRating": "ProductRisk"})

transactions_merge = transactions_merge.merge(
    tran_master.set_index("TranTypeCode")["RiskRating"],
    right_index=True,
    left_on ="TranType",
    how="left"
).rename(columns={"RiskRating": "TranTypeRisk"})

transactions_merge = transactions_merge.merge(
    currency_master.set_index("CurrencyCode")[["RiskRating"]],
    left_on="Currency",
    right_index=True,
    how="left"
).rename(columns={"RiskRating": "CurrencyRisk"})

watchlist_names = set(watchlist["Name"])

# Add WL_IC flag
transactions_merge["WL_IC"] = transactions_merge["ICName"].apply(
    lambda x: "Y" if x in watchlist_names else "N"
)

# Add WL_CP flag
transactions_merge["WL_CP"] = transactions_merge["CounterpartyName"].apply(
    lambda x: "Y" if x in watchlist_names else "N"
)

In [23]:
#Check the transaction fields after merging the other relevant fields
transactions_merge.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11000 entries, 0 to 10999
Data columns (total 35 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   TransactionDate     11000 non-null  object 
 1   TransactionID       11000 non-null  object 
 2   InitiatingCustomer  11000 non-null  object 
 3   AccountNumber       11000 non-null  object 
 4   Counterparty        11000 non-null  object 
 5   TranType            11000 non-null  object 
 6   Channel             11000 non-null  object 
 7   Currency            11000 non-null  object 
 8   Credit/Debit        11000 non-null  object 
 9   Amount              11000 non-null  float64
 10  Amount in LCY       11000 non-null  float64
 11  Product             11000 non-null  object 
 12  InitiatingGeo       11000 non-null  object 
 13  CounterpartyGeo     11000 non-null  object 
 14  BusinessCode_IC     11000 non-null  int64  
 15  CustTypeCode_IC     11000 non-null  object 
 16  PEP_

# Assign appropriate Risk levels for the relevant columns in the transaction file

In [24]:
#Replace the Risk levels with numerics (LR-LowRisk, MR-MediumRisk, HR-HighRisk with 1,5 and 10 respectively)
risk_map = {"LR": 1, "MR": 5, "HR": 10}
risk_columns = [col for col in transactions_merge.columns if col.endswith("Risk")]
transactions_merge[risk_columns] = transactions_merge[risk_columns].replace(risk_map)

/tmp/ipython-input-3666971339.py:4: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  transactions_merge[risk_columns] = transactions_merge[risk_columns].replace(risk_map)


In [25]:
transactions_features_hr = transactions_merge.iloc[:,[0,1,2,10,16,20,23,24,25,26,27,28,29,30,31,32,33,34]]

In [26]:
transactions_features_hr.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11000 entries, 0 to 10999
Data columns (total 18 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   TransactionDate     11000 non-null  object 
 1   TransactionID       11000 non-null  object 
 2   InitiatingCustomer  11000 non-null  object 
 3   Amount in LCY       11000 non-null  float64
 4   PEP_Flag_IC         11000 non-null  object 
 5   PEP_Flag_CP         11000 non-null  object 
 6   IC_BusinessRisk     11000 non-null  int64  
 7   CP_BusinessRisk     11000 non-null  int64  
 8   ChannelRisk         11000 non-null  int64  
 9   IC_GeoRisk          11000 non-null  int64  
 10  CP_GeoRisk          11000 non-null  int64  
 11  IC_CustTypeRisk     11000 non-null  int64  
 12  CP_CustTypeRisk     11000 non-null  int64  
 13  ProductRisk         11000 non-null  int64  
 14  TranTypeRisk        11000 non-null  int64  
 15  CurrencyRisk        11000 non-null  int64  
 16  WL_I

# Apply Rule based logic and generate alerts (A-P1, A-P2) and identify high risk transactions related to alerts

In [27]:
#Calculate the Overall Transaction Risk and update the same in merged transaction file in a new field'Overall_Tranx_Risk'
conditions_tx = [
    # High Risk
    (transactions_merge["PEP_Flag_CP"] == "Y") |
    (transactions_merge["CP_BusinessRisk"] == 10) |
    (transactions_merge["CP_GeoRisk"] == 10) |
    (transactions_merge["WL_CP"] == "Y") |
    (transactions_merge["ProductRisk"] == 10) |
    (transactions_merge["CurrencyRisk"] == 10) |
    (transactions_merge["ChannelRisk"] == 10),

    # Medium Risk
    (transactions_merge["PEP_Flag_CP"] == "N") &
    (transactions_merge["WL_CP"] == "N") &
    (
        (transactions_merge["CP_BusinessRisk"] == 5) |
        (transactions_merge["CP_GeoRisk"] == 5) |
        (transactions_merge["ProductRisk"] == 5) |
        (transactions_merge["CurrencyRisk"] == 5) |
        (transactions_merge["ChannelRisk"] == 5)
    ),

    # Low Risk
    (transactions_merge["PEP_Flag_CP"] == "N") &
    (transactions_merge["WL_CP"] == "N") &
    (
        (transactions_merge["CP_BusinessRisk"] == 1) |
        (transactions_merge["CP_GeoRisk"] == 1) |
        (transactions_merge["ProductRisk"] == 1) |
        (transactions_merge["CurrencyRisk"] == 1) |
        (transactions_merge["ChannelRisk"] == 1)
    )
]

choices_tx = ["HighRisk", "MedRisk", "LowRisk"]

transactions_merge["Overall_Tranx_Risk"] = np.select(conditions_tx, choices_tx, default="Unknown")

In [28]:
#Calculate the Initiating Customer overall Risk and update the same in merged transaction file in a new field'Overall_IC_Risk'
conditions_ic = [
    # High Risk
    (transactions_merge["PEP_Flag_IC"] == "Y") |
    (transactions_merge["IC_BusinessRisk"] == 10) |
    (transactions_merge["IC_GeoRisk"] == 10) |
    (transactions_merge["WL_IC"] == "Y"),

    # Medium Risk
    (transactions_merge["PEP_Flag_IC"] == "N") &
    (transactions_merge["WL_IC"] == "N") &
    ((transactions_merge["IC_BusinessRisk"] == 5) | (transactions_merge["IC_GeoRisk"] == 5)),

    # Low Risk
    (transactions_merge["PEP_Flag_IC"] == "N") &
    (transactions_merge["WL_IC"] == "N") &
    ((transactions_merge["IC_BusinessRisk"] == 1) | (transactions_merge["IC_GeoRisk"] == 1))
]

choices_ic = ["HighRisk", "MedRisk", "LowRisk"]

transactions_merge["Overall_IC_Risk"] = np.select(conditions_ic, choices_ic, default="Unknown")

In [29]:
#Check the merged transaction dataframe for the new fields
transactions_merge.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11000 entries, 0 to 10999
Data columns (total 37 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   TransactionDate     11000 non-null  object 
 1   TransactionID       11000 non-null  object 
 2   InitiatingCustomer  11000 non-null  object 
 3   AccountNumber       11000 non-null  object 
 4   Counterparty        11000 non-null  object 
 5   TranType            11000 non-null  object 
 6   Channel             11000 non-null  object 
 7   Currency            11000 non-null  object 
 8   Credit/Debit        11000 non-null  object 
 9   Amount              11000 non-null  float64
 10  Amount in LCY       11000 non-null  float64
 11  Product             11000 non-null  object 
 12  InitiatingGeo       11000 non-null  object 
 13  CounterpartyGeo     11000 non-null  object 
 14  BusinessCode_IC     11000 non-null  int64  
 15  CustTypeCode_IC     11000 non-null  object 
 16  PEP_

In [30]:
#Check the spread of initiating customers Risk profile
transactions_merge["Overall_IC_Risk"].value_counts()

,count
Overall_IC_Risk,
LowRisk,10479
HighRisk,482
MedRisk,39


In [31]:
#Check the spread of transactions Risk profile
transactions_merge["Overall_Tranx_Risk"].value_counts()

,count
Overall_Tranx_Risk,
LowRisk,6417
HighRisk,2913
MedRisk,1670


In [32]:
#Apply the rule based logic on the merged transaction file (Rule based logic is explained in the document)
#Rule based logic - Step 1 - aggregate the transaction amount by Overall_IC_Risk and Overall_Tranx_Risk
transactions_merge["TransactionID"] = transactions_merge["TransactionID"].astype(str)
agg_df = transactions_merge.groupby(
    ["InitiatingCustomer", "Overall_IC_Risk", "Overall_Tranx_Risk"]
).agg({
    "TransactionID": lambda x: ",".join(x),          # Join IDs with commas
    "Amount in LCY": "sum"                           # Total transaction amount
}).reset_index()

agg_df.rename(columns={
    "TransactionID": "Transaction IDs",
    "Amount in LCY": "Total Amount"
}, inplace=True)

# Add 'Total Tranx' column by counting commas + 1
agg_df["Total Tranx"] = agg_df["Transaction IDs"].apply(lambda x: len(x.split(",")))

print(agg_df.head())

  InitiatingCustomer Overall_IC_Risk Overall_Tranx_Risk  \
0           CUST2001        HighRisk           HighRisk   
1           CUST2001        HighRisk            LowRisk   
2           CUST2001        HighRisk            MedRisk   
3           CUST2002        HighRisk           HighRisk   
4           CUST2002        HighRisk            LowRisk   

                                     Transaction IDs  Total Amount  \
0  T20349,T20524,T20784,T21256,T21413,T22149,T233...      91187.89   
1  T20364,T20778,T21152,T21407,T23612,T24755,T255...     188057.04   
2                                             T23733        550.36   
3          T21580,T23015,T23034,T26384,T28195,T30006      16015.44   
4  T20897,T21207,T21634,T21777,T21842,T22157,T223...     186299.48   

   Total Tranx  
0           14  
1           10  
2            1  
3            6  
4           23  


In [33]:
#Rule based logic - Step 2 - Flag the alerts based on thresholds
risk_summary = transactions_merge.groupby(
    ['InitiatingCustomer', 'Overall_Tranx_Risk']
)['Amount in LCY'].sum().unstack(fill_value=0).reset_index()

# Ensure all risk levels are represented (fill if missing)
for level in ['HighRisk', 'MedRisk', 'LowRisk']:
    if level not in risk_summary.columns:
        risk_summary[level] = 0

# Step 2: Add total transaction amount per customer (T)
risk_summary['TotalAmount'] = (
    risk_summary['HighRisk'] + risk_summary['MedRisk'] + risk_summary['LowRisk']
)

# Step 3: Compute H/T and M/T ratios
risk_summary['H_ratio'] = risk_summary['HighRisk'] / risk_summary['TotalAmount']
risk_summary['M_ratio'] = risk_summary['MedRisk'] / risk_summary['TotalAmount']

# Step 4: Prepare a mapping from InitiatingCustomer to Alert value
def determine_alert(row):
    if row['H_ratio'] >= 0.80 and row['HighRisk'] >= 1500:
        return 'A-P1'
    elif row['M_ratio'] >= 0.60 and row['MedRisk'] >= 1000:
        return 'A-P2'
    else:
        return 'N'

risk_summary['Alert'] = risk_summary.apply(determine_alert, axis=1)

# Step 5: Merge the Alert info back into agg_df
agg_df = agg_df.merge(
    risk_summary[['InitiatingCustomer', 'Alert']],
    on='InitiatingCustomer',
    how='left'
)

In [34]:
agg_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1474 entries, 0 to 1473
Data columns (total 7 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   InitiatingCustomer  1474 non-null   object 
 1   Overall_IC_Risk     1474 non-null   object 
 2   Overall_Tranx_Risk  1474 non-null   object 
 3   Transaction IDs     1474 non-null   object 
 4   Total Amount        1474 non-null   float64
 5   Total Tranx         1474 non-null   int64  
 6   Alert               1474 non-null   object 
dtypes: float64(1), int64(1), object(5)
memory usage: 80.7+ KB


In [35]:
risk_summary[risk_summary['Alert']=='A-P2']

Overall_Tranx_Risk,InitiatingCustomer,HighRisk,LowRisk,MedRisk,TotalAmount,H_ratio,M_ratio,Alert
29,CUST2030,7698.65,51667.61,91427.21,150793.47,0.051054,0.606307,A-P2
60,CUST2061,20009.93,40873.16,137615.84,198498.93,0.100806,0.693283,A-P2
100,CUST2101,10636.35,65287.36,115959.93,191883.64,0.055431,0.604324,A-P2
133,CUST2134,9751.51,44251.35,220118.71,274121.57,0.035574,0.802997,A-P2
192,CUST2193,11101.06,37557.60,83013.76,131672.42,0.084308,0.630457,A-P2
279,CUST2280,11895.18,47439.42,95794.37,155128.97,0.076679,0.617514,A-P2
360,CUST2361,9401.94,74056.63,236857.78,320316.35,0.029352,0.739450,A-P2
383,CUST2384,2271.47,32437.61,101442.57,136151.65,0.016683,0.745070,A-P2


In [36]:
risk_summary[risk_summary['Alert']=='A-P1']

Overall_Tranx_Risk,InitiatingCustomer,HighRisk,LowRisk,MedRisk,TotalAmount,H_ratio,M_ratio,Alert
78,CUST2079,198918.05,23288.86,17102.90,239309.81,0.831216,0.071468,A-P1
95,CUST2096,149928.17,36094.78,0.00,186022.95,0.805966,0.000000,A-P1
129,CUST2130,128656.86,18882.18,964.79,148503.83,0.866354,0.006497,A-P1
223,CUST2224,91031.97,18115.50,254.17,109401.64,0.832090,0.002323,A-P1
259,CUST2260,140521.49,27263.14,4408.30,172192.93,0.816070,0.025601,A-P1
302,CUST2303,129191.80,27438.89,37.92,156668.61,0.824618,0.000242,A-P1
358,CUST2359,97747.76,17251.94,0.00,114999.70,0.849983,0.000000,A-P1
390,CUST2391,174383.74,25331.23,2022.79,201737.76,0.864408,0.010027,A-P1


In [37]:
agg_df['Alert'].value_counts()

,count
Alert,
N,1428
A-P2,24
A-P1,22


In [38]:
#Rule based logic - Step 3 - Identify the transactions related to the alerts
# Step 1: Filter agg_df for alerts A-P1 and A-P2
filtered_agg_df = agg_df[agg_df['Alert'].isin(['A-P1', 'A-P2'])].copy()

# Step 2: Split 'Transaction IDs' string into a list
filtered_agg_df['Transaction IDs'] = filtered_agg_df['Transaction IDs'].str.split(',')

# Step 3: Explode into individual rows
flat_trx_df = filtered_agg_df.explode('Transaction IDs').reset_index(drop=True)

# Step 4: Trim whitespaces if any
flat_trx_df['Transaction IDs'] = flat_trx_df['Transaction IDs'].str.strip()

# Step 5: Keep only needed columns
flat_trx_df = flat_trx_df[['InitiatingCustomer', 'Transaction IDs', 'Alert']]

In [39]:
flat_trx_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 325 entries, 0 to 324
Data columns (total 3 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   InitiatingCustomer  325 non-null    object
 1   Transaction IDs     325 non-null    object
 2   Alert               325 non-null    object
dtypes: object(3)
memory usage: 7.7+ KB


In [40]:
flat_trx_df["Alert"].value_counts()

,count
Alert,
A-P2,176
A-P1,149


#Apply VAE model on the new data and identify high risk transactions

* Use the VAE model already built and trained  

In [41]:
import tensorflow as tf
from tensorflow.keras import layers, Model
import keras

In [42]:
@keras.saving.register_keras_serializable()
class VAE(Model):
    def __init__(self, input_dim, latent_dim, **kwargs):
        super().__init__(**kwargs)
        self.input_dim = input_dim
        self.latent_dim = latent_dim

        self.encoder = tf.keras.Sequential([
            layers.Input(shape=(input_dim,)),
            layers.Dense(512, activation='relu'),
            layers.Dense(256, activation='relu'),
            layers.Dense(128, activation='relu'),
            layers.Dense(64, activation='relu'),
            layers.Dense(32, activation='relu'),
            layers.Dense(16, activation='relu'),
            layers.Dense(8, activation='relu'),
            layers.Dense(latent_dim * 2),  # z_mean and z_log_var
        ])

        self.decoder = tf.keras.Sequential([
            layers.Input(shape=(latent_dim,)),
            layers.Dense(8, activation='relu'),
            layers.Dense(16, activation='relu'),
            layers.Dense(32, activation='relu'),
            layers.Dense(64, activation='relu'),
            layers.Dense(128, activation='relu'),
            layers.Dense(256, activation='relu'),
            layers.Dense(512, activation='relu'),
            layers.Dense(input_dim, activation='sigmoid'),
        ])

    def sample(self, z_mean, z_log_var):
        eps = tf.random.normal(shape=tf.shape(z_mean))
        return z_mean + tf.exp(0.5 * z_log_var) * eps

    def call(self, inputs):
        # Encode
        z_params = self.encoder(inputs)
        z_mean, z_log_var = tf.split(z_params, num_or_size_splits=2, axis=1)
        z = self.sample(z_mean, z_log_var)

        # Decode
        reconstructed = self.decoder(z)

        # Compute VAE Loss
        reconstruction_loss = tf.reduce_mean(tf.square(inputs - reconstructed), axis=1)
        kl_loss = -0.5 * tf.reduce_mean(
            1 + z_log_var - tf.square(z_mean) - tf.exp(z_log_var), axis=1
        )
        total_loss = tf.reduce_mean(reconstruction_loss + kl_loss)
        self.add_loss(total_loss)

        return reconstructed

    def get_config(self):
        config = super().get_config()
        config.update({
            "input_dim": self.input_dim,
            "latent_dim": self.latent_dim
        })
        return config

In [43]:
vae_loaded = tf.keras.models.load_model(
    "vae.keras",
    compile=False
    )

In [44]:
transactions_features = transactions_merge.iloc[:,[10,20,24,25,27,30,32,34]]

In [45]:
transactions_features.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11000 entries, 0 to 10999
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Amount in LCY    11000 non-null  float64
 1   PEP_Flag_CP      11000 non-null  object 
 2   CP_BusinessRisk  11000 non-null  int64  
 3   ChannelRisk      11000 non-null  int64  
 4   CP_GeoRisk       11000 non-null  int64  
 5   ProductRisk      11000 non-null  int64  
 6   CurrencyRisk     11000 non-null  int64  
 7   WL_CP            11000 non-null  object 
dtypes: float64(1), int64(5), object(2)
memory usage: 687.6+ KB


In [46]:
all_indices = np.arange(len(transactions_features))

# Just use all data for prediction
X_all_raw = transactions_features
idx_all = all_indices


In [47]:
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
import matplotlib.pyplot as plt
import seaborn as sns
import keras

# Step 1: Define feature groups
numeric_features = ["Amount in LCY", "CP_BusinessRisk","ChannelRisk", "CP_GeoRisk", "ProductRisk","CurrencyRisk"]

binary_features = ["PEP_Flag_CP", "WL_CP"]

# Step 2: Define transformers
preprocessor = ColumnTransformer(
    transformers=[
        ("num", MinMaxScaler(), numeric_features),
        ("bin", OneHotEncoder(drop='if_binary', dtype=int), binary_features)
    ]
)

# Step 3: Set up pipeline
pipeline = Pipeline(steps=[("preprocessor", preprocessor)])
#X_preprocessed = pipeline.fit_transform(transactions_features_hr_vae)
pipeline.fit(X_all_raw)
X_all = pipeline.transform(X_all_raw)


In [49]:
reconstructed = vae_loaded.predict(X_all)
reconstruction_error = tf.reduce_mean(tf.square(X_all - reconstructed), axis=1).numpy()
threshold = np.percentile(reconstruction_error, 97)

anomalies = reconstruction_error > threshold
print(f"Anomalies detected: {np.sum(anomalies)} / {len(X_all)}")

344/344 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
Anomalies detected: 330 / 11000


In [50]:
# Get indices of anomalies
anomaly_indices = np.where(anomalies)[0]

# View the actual anomalous transaction(s)
anomalous_transactions = X_all[anomaly_indices]

In [51]:
print("Anomaly found at row(s):", anomaly_indices)

Anomaly found at row(s): [   68    69    91   117   172   184   258   265   328   349   374   406
   411   417   452   460   501   601   602   624   672   733   737   784
   820   822   844   845   864   888   948  1010  1057  1067  1071  1075
  1094  1120  1148  1149  1157  1170  1200  1209  1281  1345  1368  1375
  1416  1455  1490  1521  1526  1546  1567  1580  1588  1591  1653  1668
  1695  1723  1726  1823  1856  1913  1951  1994  2043  2046  2057  2103
  2155  2186  2202  2237  2289  2317  2346  2376  2390  2411  2416  2460
  2467  2495  2564  2636  2638  2705  2727  2747  2766  2865  2955  2959
  3054  3073  3079  3130  3190  3263  3276  3310  3353  3381  3399  3445
  3544  3563  3567  3674  3691  3704  3707  3717  3849  3868  3906  3917
  3956  3980  4029  4033  4056  4071  4073  4085  4086  4120  4126  4186
  4267  4271  4332  4334  4371  4489  4491  4542  4610  4611  4714  4738
  4778  4805  4815  4828  4945  4961  4969  4970  5061  5116  5170  5252
  5265  5337  5379  5404  

In [52]:
# Step 3: Get global indices (in original DataFrame)
global_anomaly_indices = idx_all[anomaly_indices]

# Step 4: Fetch those rows from transactions_features_hr
anomaly_transactions_df = transactions_merge.iloc[global_anomaly_indices]

# Optional: View it
import pandas as pd
pd.set_option('display.max_columns', None)
display(anomaly_transactions_df)

,TransactionDate,TransactionID,InitiatingCustomer,AccountNumber,Counterparty,TranType,Channel,Currency,Credit/Debit,Amount,Amount in LCY,Product,InitiatingGeo,CounterpartyGeo,BusinessCode_IC,CustTypeCode_IC,PEP_Flag_IC,ICName,BusinessCode_CP,CustTypeCode_CP,PEP_Flag_CP,CounterpartyName,ProductCode,IC_BusinessRisk,CP_BusinessRisk,ChannelRisk,IC_GeoRisk,CP_GeoRisk,IC_CustTypeRisk,CP_CustTypeRisk,ProductRisk,TranTypeRisk,CurrencyRisk,WL_IC,WL_CP,Overall_Tranx_Risk,Overall_IC_Risk
68,2025-08-05,T20068,CUST2372,ACC02372,CUST2357,FTIN,IBAN,GBP,Cr,85556.72,106945.90,OVDF,AUT,KEN,1015,NGOS,N,Julie Houston Non-Govt Org,1007,TRST,N,Christopher Casey Trust Co,OVDF,1,1,10,1,5,1,1,1,10,1,N,N,HighRisk,LowRisk
69,2025-08-11,T20069,CUST2418,ACC02527,CUST2409,LREP,IBAN,DKK,Cr,624.07,93.61,LAPR,ARG,BRA,1017,PVTL,N,Stephanie Barber Pvt Ltd Company,1006,PVTL,N,James Smith Pvt Ltd Company,BILLS,1,1,10,1,1,1,1,1,1,1,N,Y,HighRisk,LowRisk
91,2025-08-10,T20091,CUST2398,ACC02640,CUST2395,FTIN,IBAN,EUR,Cr,32953.32,36248.65,SMEL,ARE,NPL,1024,NGOS,N,Hannah Cohen Non-Govt Org,1011,INDI,N,Jerry Carrillo,SMEL,1,1,10,1,10,1,1,1,10,1,N,N,HighRisk,LowRisk
117,2025-08-05,T20117,CUST2411,ACC02411,CUST2454,FTIN,IBAN,SGD,Cr,1694.77,1321.92,BUSL,COL,ARE,1022,BFIS,N,Alexis Martinez Banking Ltd,1019,PUBL,N,Sabrina Buckley Pub Ltd Company,AUTL,1,1,10,1,1,1,1,10,10,1,N,N,HighRisk,LowRisk
172,2025-08-03,T20172,CUST2314,ACC02887,CUST2340,FTIN,BRAN,USD,Cr,14455.36,14455.36,BUSL,BRA,NPL,1025,PUBL,N,Heather Holt Pub Ltd Company,1017,MBNK,N,Isaac Stone Banking Ltd,BILLS,1,1,1,1,10,1,1,10,10,1,N,N,HighRisk,LowRisk
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10834,2025-08-09,T30834,CUST2127,ACC02127,CUST2331,FTDO,CHEQ,VEF,Cr,493.81,39.50,LAPR,USD,USD,1022,NGOS,N,Chad Mercer Non-Govt Org,1005,INDI,Y,Robert Calderon,ELON,1,1,1,1,1,1,1,1,5,10,N,N,HighRisk,LowRisk
10902,2025-08-03,T30902,CUST2136,ACC03069,CUST2020,MPAY,IBAN,EUR,Cr,749.05,823.96,PLON,GBR,CHN,1016,INDI,N,Gregory Jackson,1007,NBFC,N,Jason Hayes Non-Banking Fin Co,OVDF,1,1,10,1,1,1,1,1,1,1,N,Y,HighRisk,LowRisk
10934,2025-08-07,T30934,CUST2141,ACC02992,CUST2448,FTIN,IBAN,EUR,Cr,997.21,1096.93,PLON,AUT,NPL,1007,LLPF,N,Ann Salazar Limited LP,1006,FCOM,N,Denise Warren Foreign Co,FIXD,1,1,10,1,10,1,1,1,10,1,Y,N,HighRisk,HighRisk
10958,2025-08-15,T30958,CUST2234,ACC02562,CUST2437,FTIN,SELF,USD,Cr,53596.69,53596.69,BUSL,CHN,NPL,1001,NGOS,N,Manuel Santos Non-Govt Org,1022,PVTL,N,Ronald Nunez Pvt Ltd Company,BILLS,1,1,1,1,10,1,1,10,10,1,N,N,HighRisk,LowRisk


# Compare the high risk transactions flagged by Rule-based algorithm with VAE model output of anamolous transactions

In [53]:
# Step 1: Extract transaction IDs from both sources
vae_anomaly_ids = anomaly_transactions_df["TransactionID"].astype(str).unique()
rule_based_ids = flat_trx_df["Transaction IDs"].astype(str).unique()

# Step 2: Find matches (overlap) and mismatches
matched_ids = np.intersect1d(vae_anomaly_ids, rule_based_ids)
vae_only_ids = np.setdiff1d(vae_anomaly_ids, rule_based_ids)
rule_only_ids = np.setdiff1d(rule_based_ids, vae_anomaly_ids)

# Step 3: Print results
print(f"Total VAE anomalies: {len(vae_anomaly_ids)}")
print(f"Total Rule-based alerts: {len(rule_based_ids)}")
print(f"Matched (VAE ∩ Rule-based): {len(matched_ids)}")
print(f"Match % from VAE anomalies: {len(matched_ids) / len(vae_anomaly_ids) * 100:.2f}%")

# Step 4 (Optional): Get full details of matches and mismatches
matched_df = anomaly_transactions_df[anomaly_transactions_df["TransactionID"].isin(matched_ids)]
vae_only_df = anomaly_transactions_df[anomaly_transactions_df["TransactionID"].isin(vae_only_ids)]
rule_only_tranx = flat_trx_df[flat_trx_df["Transaction IDs"].isin(rule_only_ids)]
rule_only_tranx = rule_only_tranx.rename(columns={'Transaction IDs': 'TransactionID'})
rule_only_df = rule_only_tranx.merge(transactions_merge, on='TransactionID', how='left')

Total VAE anomalies: 330
Total Rule-based alerts: 325
Matched (VAE ∩ Rule-based): 12
Match % from VAE anomalies: 3.64%
